** We are going to be working through understanding and interacting with the PCP Plugin framework.  The "Sample Plugin is copied below from the prior "SamplePLugin" notebook. **

** The two core functions will be to: **
* Request jobs
* Respond with job output

** The start function should be an infinite loop handling these two processes. **
** The Logger is an object that you use to provide log information to central storage. **
** The ext_signal is an external signal that will need to be checked within your process loop.  When a "TRUE" signal is received, the framework expects the start function to return. **
** The functionality will need to be a list of valid commands. If you have built the commands for your plugin, from the Introduction.ipynb, copy and paste these commands into this notebook. **
** WARNING:  The framework may terminate your thread. **

** The sample command is pasted below and the command variable (cmdWget) is pasted into your SamplePlugin Class to demonstrate building the command(s) into your custom plugin. **

In [ ]:
try:
    from src.controller_plugin import ControllerPlugin
except ImportError:
    from sys import path
    path.append("/root/backend-interpreter/plugin_interpreter")
    print(path)
    from src.controller_plugin import ControllerPlugin
    from os import environ
    environ["PORT"] = "10000" 

In [ ]:
cmdWget = {"CommandName" : "wget",
          "Tooltip" : "Retrieves raw webpage files from targeted systems",
          "Output" : True,
          "Inputs" : [{"Name" : "Target IP Address",
                     "Type" : "url",
                     "Tooltip" : "Modify the URL to the desired target IP address",
                     "Value" : "http://192.168.1.100"}]}

In [ ]:
class SamplePlugin(ControllerPlugin):
    functionality =  [cmdWget]                         # Get the commands you built from the prior tutorial
    def __init__(self):
        """
        The plugin class is the last class to init,
        but Database and logger have not started yet
        Should not be using the database handles or the logger in this fn
        """
        self.name = "SamplePlugin"
        super().__init__(self.name, self.functionality)
    def start(self, logger, ext_signal):
        """
        Start function is called by the supervisor only
        This function may assume the logger and the db is already start()ed
        This function should include all calls required to run until ext_signal says to stop.

        :param logger: <central logger> object for all log handling
        :param ext_signal: <ctypes.bool> the kill signal
        :return: This function calls exit! when complete
        """
        #################################################################################################
        # Add Plugin specific startup routine here
        
        pass
        ################################################################################################
        # Add plugin control loop here
        while not ext_signal.value:                                 # As discussed in section 3 above
            # Assume you have a connected client for this controller
            new_job = self._request_job()  # <dict> or None  
            if not None:  # use your plugin functionality to get this job to your connected client
                output_content = None # or <add code for the plugin content for this target/job (string obj)
                # Give this content back to the framework
                self._respond_output(job, output_content)
            sleep(3)                                        
        
        ###############################################################################################
        # Put cleanup code here
        
        pass
        ###############################################################################################
        
    def _stop(self):
        pass

In [ ]:
SamplePlugin()